In [5]:
import numpy as np, pandas as pd
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
sys.path.append('..')
from nestedcvtraining.under_sampling_classifier import UnderSamplingClassifier
from nestedcvtraining.switch_case import SwitchCase 
from nestedcvtraining.api import find_best_model 
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline as SkPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer, fbeta_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from skopt.space import Real, Integer, Categorical
from skopt import gbrt_minimize

dataset_prueba = pd.read_csv(
    "https://raw.githubusercontent.com/JaimeArboleda/nestedcvtraining/master/datasets/new-thyroid.csv", header=None
)
values = dataset_prueba.values
X, y = values[:, :-1], (values[:, -1] - 1).astype(int)

resampler = SwitchCase(
    cases=[
        (
            "resampler_1",
            SMOTE(k_neighbors=3)
        ),
        (
            "resampler_2",
            "passthrough"
        )
    ],
    switch="resampler_1"
)

preprocessor = SwitchCase(
    cases=[
        (
            "prep_1",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", PCA(n_components=5))
            ])
        ),
        (
            "prep_2",
            SkPipeline([
                ("scale", StandardScaler()), 
                ("reduce_dims", SelectKBest(mutual_info_classif, k=5)),
            ])
        ),
        (
            "prep_3",
            "passthrough"
        )
    ],
    switch="prep_1"
)

model = SwitchCase(
    cases=[
        (
            "model_1",
            LogisticRegression()
        ),
        (
            "model_2",
            RandomForestClassifier()
        )
    ],
    switch="model_1"
)

clf = ImbPipeline(
    [("resampler", resampler), ("preprocessor", preprocessor), ("model", model)]
)

search_space= [
    Categorical(["resampler_1", "resampler_2"], name="resampler__switch"),
    Categorical(["prep_1", "prep_2", "prep_3"], name="preprocessor__switch"),
    Categorical(["model_1", "model_2"], name="model__switch"),
    Categorical(["minority", "all"], name="resampler__resampler_1__sampling_strategy"),
    Integer(5, 15, name="model__model_2__max_depth")
]

best_model, report = find_best_model(
    X=X,
    y=y,
    model=clf,
    search_space=search_space,
    verbose=True,
    k_inner=6,
    k_outer=6,
    skip_inner_folds=[1, 3, 5],
    skip_outer_folds=[1, 3, 5],
    n_initial_points=15,
    n_calls=20,
    calibrate="only_best",
    calibrate_params={"method": "isotonic"},
    optimizing_metric=make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True),
    other_metrics={"acc": "accuracy"},
    skopt_func=gbrt_minimize
)

Looping over 0 outer fold
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_1', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 7}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_2', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 9}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_1', 'model__switch': 'model_1', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 6}
Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'all', 'model__model_2__max_depth': 9}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': '

/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 6}
Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_1', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 9}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_2', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'all', 'model__model_2__max_depth': 9}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_1', 'resampler__resampler_1__sampling_strategy': 'all', 'model__model_2__max_depth': 7}


/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_1', 'resampler__resampler_1__sampling_strategy': 'all', 'model__model_2__max_depth': 15}


...

/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 9}
Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_1', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 6}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 12}


/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 11}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_3', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 13}
Optimizing model with parameters {'resampler__switch': 'resampler_1', 'preprocessor__switch': 'prep_1', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 12}
Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_1', 'model__switch': 'model_2', 'resampler__resampler_1__sampling_strategy': 'minority', 'model__model_2__max_depth': 10}
Optimizing model with parameters {'resampler__switch': 'resampler_2', 'preprocessor__switch': 'prep_1', 'model__

/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jaime/projects/python/nestedcvtraining/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

In [6]:
best_model



CalibratedClassifierCV(base_estimator=Pipeline(steps=[('resampler',
                                                       SwitchCase(cases=[('resampler_1',
                                                                          SMOTE(k_neighbors=3,
                                                                                sampling_strategy='minority')),
                                                                         ('resampler_2',
                                                                          'passthrough')],
                                                                  switch='resampler_2')),
                                                      ('preprocessor',
                                                       SwitchCase(cases=[('prep_1',
                                                                          Pipeline(steps=[('scale',
                                                                                           StandardScaler()),
                                                                                          ('reduce_dims',
                                                                                           PCA(n_components=5))])),
                                                                         ('prep_2',
                                                                          Pipeline(steps=[('scale',
                                                                                           StandardScaler()),
                                                                                          ('reduce_dims',
                                                                                           SelectKBest(k=5,
                                                                                                       score_func=<function mutual_info_classif at 0x7fc8b3048820>))])),
                                                                         ('prep_3',
                                                                          'passthrough')],
                                                                  switch='prep_3')),
                                                      ('model',
                                                       SwitchCase(cases=[('model_1',
                                                                          LogisticRegression()),
                                                                         ('model_2',
                                                                          RandomForestClassifier(max_depth=15))],
                                                                  switch='model_1'))]),
                       method='isotonic')

In [7]:
report.to_dataframe()

,best,outer_kfold,model,outer_test_indexes,param__model__model_2__max_depth,param__model__switch,param__preprocessor__switch,param__resampler__resampler_1__sampling_strategy,param__resampler__switch,inner_validation_metrics__optimizing_metric,inner_validation_metrics__acc,outer_test_metrics__optimizing_metric,outer_test_metrics__acc
0,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",7,model_2,prep_1,minority,resampler_1,0.972937,0.965900,NaN,NaN
1,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",9,model_2,prep_2,minority,resampler_1,0.991881,0.960536,NaN,NaN
2,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6,model_1,prep_1,minority,resampler_1,0.991848,0.960345,NaN,NaN
3,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",9,model_2,prep_3,all,resampler_2,0.997536,0.966092,NaN,NaN
4,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",11,model_1,prep_1,minority,resampler_1,0.989745,0.960345,NaN,NaN
5,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",8,model_2,prep_1,minority,resampler_1,0.975144,0.949042,NaN,NaN
6,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,model_2,prep_3,all,resampler_1,0.987768,0.966284,NaN,NaN
7,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",5,model_2,prep_1,all,resampler_2,0.992038,0.949234,NaN,NaN
8,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",10,model_1,prep_2,all,resampler_2,0.991367,0.965900,NaN,NaN
9,False,0,None,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",12,model_1,prep_2,all,resampler_2,0.991367,0.965900,NaN,NaN


In [11]:
resampler.fit_resample(X, y, **{})

array([[107. ,  10.1,   2.2,   0.9,   2.7],
       [113. ,   9.9,   3.1,   2. ,   5.9],
       [127. ,  12.9,   2.4,   1.4,   0.6],
       ...,
       [103. ,   5.1,   1.4,   1.2,   5. ],
       [ 97. ,   4.7,   1.1,   2.1,  12.6],
       [102. ,   5.3,   1.4,   1.3,   6.7]])

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
clf.predict_proba(X)

array([[0.98 , 0.01 , 0.01 ],
       [0.75 , 0.136, 0.114],
       [0.836, 0.12 , 0.044],
       [0.546, 0.   , 0.454],
       [0.896, 0.076, 0.028],
       [0.52 , 0.004, 0.476],
       [0.994, 0.   , 0.006],
       [0.88 , 0.006, 0.114],
       [0.884, 0.108, 0.008],
       [0.894, 0.072, 0.034],
       [0.396, 0.   , 0.604],
       [0.966, 0.024, 0.01 ],
       [0.886, 0.092, 0.022],
       [0.768, 0.008, 0.224],
       [0.966, 0.   , 0.034],
       [0.978, 0.012, 0.01 ],
       [0.988, 0.01 , 0.002],
       [0.894, 0.   , 0.106],
       [0.95 , 0.018, 0.032],
       [0.924, 0.   , 0.076],
       [0.96 , 0.004, 0.036],
       [0.87 , 0.12 , 0.01 ],
       [0.828, 0.17 , 0.002],
       [0.946, 0.002, 0.052],
       [0.88 , 0.092, 0.028],
       [0.692, 0.028, 0.28 ],
       [0.968, 0.016, 0.016],
       [0.97 , 0.012, 0.018],
       [0.95 , 0.048, 0.002],
       [0.998, 0.002, 0.   ],
       [0.922, 0.006, 0.072],
       [0.976, 0.018, 0.006],
       [0.788, 0.006, 0.206],
       [0.

In [7]:
clf.estimators_

[RandomForestClassifier(),
 RandomForestClassifier(),
 RandomForestClassifier(),
 RandomForestClassifier(),
 RandomForestClassifier()]